### 2020 KMU D&A Machine Learning Session Week 2

In [1]:
import pandas as pd 
import numpy as np
import warnings

warnings.filterwarnings(action='ignore')

#### 데이터 불러오기

In [2]:
cust_tr = pd.read_csv('data/w2/cust_train.csv', encoding = 'utf-8') #회원
product_tr = pd.read_csv('data/w2/product_train.csv',encoding = 'utf-8') #상품구매
search_tr = pd.read_csv('data/w2/search_train.csv',encoding = 'utf-8') #검색어
session_tr = pd.read_csv('data/w2/session_train.csv',encoding = 'utf-8') #세션

In [3]:
cust_te = pd.read_csv('data/w2/cust_test.csv', encoding = 'utf-8')
product_te = pd.read_csv('data/w2/product_test.csv',encoding = 'utf-8')
search_te = pd.read_csv('data/w2/search_test.csv',encoding = 'utf-8')
session_te = pd.read_csv('data/w2/session_test.csv',encoding = 'utf-8')

In [4]:
master = pd.read_csv('data/w2/master.csv', encoding = 'utf-8') #상품분류

#### 데이터 전처리

In [5]:
train = pd.merge(product_tr, session_tr, on = ['CLNT_ID','SESS_ID'])
train = pd.merge(train, search_tr, on = ['CLNT_ID','SESS_ID'])
train = pd.merge(train, master, on = 'PD_C')

In [6]:
test = pd.merge(product_te, session_te, on = ['CLNT_ID','SESS_ID'])
test = pd.merge(test, search_te, on = ['CLNT_ID','SESS_ID'])
test = pd.merge(test, master, on = 'PD_C')

In [7]:
train['PD_BUY_AM']  = train['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
train['PD_BUY_CT']  = train['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_AM']  = test['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_CT']  = test['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )

#### Feature 생성

In [8]:
features = []
features_te = []

In [9]:
# 총구매액, 구매건수, 평균구매액, 최대구매액
train['AMOUNT'] = train['PD_BUY_AM'] * train['PD_BUY_CT']
test['AMOUNT'] = test['PD_BUY_AM'] * test['PD_BUY_CT']

In [10]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features.append(f)

f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features_te.append(f_te)

In [12]:
# 주말방문비율
train['date'] = pd.to_datetime(train['SESS_DT'], format= '%Y%m%d')
test['date'] = pd.to_datetime(test['SESS_DT'], format= '%Y%m%d')

In [13]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features.append(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features_te.append(f_te)

In [14]:
#계절방문비율
f = train.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean( x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean( x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,12])))
]).reset_index()
features.append(f)

In [15]:
f_te = test.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean( x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean( x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,12])))
]).reset_index()
features_te.append(f_te)

#### Merge features

In [16]:
data = pd.DataFrame({'CLNT_ID': train.CLNT_ID.unique()})
for f in features :
    data = pd.merge(data, f, how='left')
    
data = data.fillna(0)

In [17]:
data_te = pd.DataFrame({'CLNT_ID': test.CLNT_ID.unique()})
for f in features_te :
    data_te = pd.merge(data_te, f, how='left')
    
data_te = data_te.fillna(0)

#### 새로운 numeric 변수 추가하기

In [18]:
# 구매주기
f = train.groupby('CLNT_ID')['date'].agg([
    ('구매주기', lambda x: int(((x.max() - x.min()).days) / x.nunique())),
]).reset_index()

In [19]:
f_te = test.groupby('CLNT_ID')['date'].agg([
    ('구매주기', lambda x: int(((x.max() - x.min()).days) / x.nunique())),
]).reset_index()

In [20]:
# 기기 유형별 구매비율
f = train.groupby('CLNT_ID')['DVC_CTG_NM'].agg([
    ('Desktop-구매비율', lambda x: np.mean(x == 'desktop')),
    ('Mobile-구매비율', lambda x: np.mean(x == 'mobile')),
    ('Tablet-구매비율', lambda x: np.mean(x == 'tablet')),
]).reset_index()

In [21]:
f_te = test.groupby('CLNT_ID')['DVC_CTG_NM'].agg([
    ('Desktop-구매비율', lambda x: np.mean(x == 'desktop')),
    ('Mobile-구매비율', lambda x: np.mean(x == 'mobile')),
    ('Tablet-구매비율', lambda x: np.mean(x == 'tablet')),
]).reset_index()